<a href="https://colab.research.google.com/github/prititaliya/Predicting-Optimal-Fertilizers/blob/main/Predicting_Optimal_Fertilizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import classification_report

In [50]:
train=pd.read_csv('/content/train_processed.csv')
test=pd.read_csv('/content/test_processed.csv')
y=pd.read_csv('/content/y.csv')

In [41]:
train_old=pd.read_csv('/content/train.csv')
test_old=pd.read_csv('/content/test.csv')

In [5]:
train=train_df.copy()
test=test_df.copy()

In [18]:
y=train.pop('Fertilizer Name')

In [42]:
train_id=train_old.pop('id')
test_id=test_old.pop('id')

In [24]:
ohe_soil=OneHotEncoder()
ohe_crop=OneHotEncoder()
ohe_soil.fit(train[['Soil Type']])
ohe_crop.fit(train[['Crop Type']])

OneHotEncoder()

In [29]:
ohe_soil_df=pd.DataFrame(ohe_soil.transform(train[['Soil Type']]).toarray(),columns=ohe_soil.get_feature_names_out())
ohe_crop_df=pd.DataFrame(ohe_crop.transform(train[['Crop Type']]).toarray(),columns=ohe_crop.get_feature_names_out())
ohe_soil_test=pd.DataFrame(ohe_soil.transform(test[['Soil Type']]).toarray(),columns=ohe_soil.get_feature_names_out())
ohe_crop_test=pd.DataFrame(ohe_crop.transform(test[['Crop Type']]).toarray(),columns=ohe_crop.get_feature_names_out())

In [30]:
train=pd.concat([train,ohe_soil_df,ohe_crop_df],axis=1)
test=pd.concat([test,ohe_soil_test,ohe_crop_test],axis=1)

In [33]:
train.drop(['Soil Type','Crop Type'],axis=1,inplace=True)
test.drop(['Soil Type','Crop Type'],axis=1,inplace=True)

In [37]:
train.sample(10)

,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil Type_Black,Soil Type_Clayey,Soil Type_Loamy,Soil Type_Red,...,Crop Type_Cotton,Crop Type_Ground Nuts,Crop Type_Maize,Crop Type_Millets,Crop Type_Oil seeds,Crop Type_Paddy,Crop Type_Pulses,Crop Type_Sugarcane,Crop Type_Tobacco,Crop Type_Wheat
316360,32,51,53,16,9,42,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
153354,31,64,29,19,4,40,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
416424,33,72,42,5,16,38,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
333953,35,52,63,27,12,17,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
634814,26,64,63,36,11,15,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
656184,31,58,35,22,8,5,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
597852,38,50,59,4,15,18,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
165697,28,62,49,30,1,15,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
291266,32,67,44,6,12,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
522272,37,60,36,29,17,5,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [38]:
plt.figure(figsize=(20,10))
sns.pairplot(train)
plt.show()

<Figure size 2000x1000 with 0 Axes>

KeyboardInterrupt: 

In [39]:
train

,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil Type_Black,Soil Type_Clayey,Soil Type_Loamy,Soil Type_Red,...,Crop Type_Cotton,Crop Type_Ground Nuts,Crop Type_Maize,Crop Type_Millets,Crop Type_Oil seeds,Crop Type_Paddy,Crop Type_Pulses,Crop Type_Sugarcane,Crop Type_Tobacco,Crop Type_Wheat
0,37,70,36,36,4,5,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,27,69,65,30,6,18,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29,63,32,24,12,16,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,35,62,54,39,12,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,35,58,43,37,2,16,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,25,69,30,8,16,6,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
749996,37,64,58,38,8,20,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
749997,35,68,59,6,11,29,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
749998,31,68,29,9,11,12,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
train_df['Soil Type'].value_counts()

,count
Soil Type,
Sandy,156710
Black,150956
Clayey,148382
Red,148102
Loamy,145850


In [43]:
train.drop(columns=['Soil Type_Black','Crop Type_Barley'],inplace=True)
test.drop(columns=['Soil Type_Black','Crop Type_Barley'],inplace=True)

In [44]:
train

,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil Type_Clayey,Soil Type_Loamy,Soil Type_Red,Soil Type_Sandy,Crop Type_Cotton,Crop Type_Ground Nuts,Crop Type_Maize,Crop Type_Millets,Crop Type_Oil seeds,Crop Type_Paddy,Crop Type_Pulses,Crop Type_Sugarcane,Crop Type_Tobacco,Crop Type_Wheat
0,37,70,36,36,4,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,27,69,65,30,6,18,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29,63,32,24,12,16,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,35,62,54,39,12,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,35,58,43,37,2,16,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,25,69,30,8,16,6,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
749996,37,64,58,38,8,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
749997,35,68,59,6,11,29,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
749998,31,68,29,9,11,12,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
train_con=train.select_dtypes(include='int64')
test_con=test.select_dtypes(include='int64')

In [82]:
scaler=StandardScaler()
scaler.fit(train_con)
train_con_scaled=scaler.transform(train_con)
test_con_scaled=scaler.transform(test_con)

In [83]:
train_con_scaled=pd.DataFrame(train_con_scaled,columns=train_con.columns)
test_con_scaled=pd.DataFrame(test_con_scaled,columns=test_con.columns)

In [85]:
train.drop(columns=train_con.columns,inplace=True)
test.drop(columns=test_con.columns,inplace=True)
train=pd.concat([train,train_con_scaled],axis=1)
test=pd.concat([test,test_con_scaled],axis=1)

In [87]:
train.to_csv('train_processed.csv',index=False)
test.to_csv('test_processed.csv',index=False)

In [31]:
le=LabelEncoder()
le.fit(train_old['Fertilizer Name'])

LabelEncoder()

In [32]:
le.classes_

array(['10-26-26', '14-35-14', '17-17-17', '20-20', '28-28', 'DAP',
       'Urea'], dtype=object)

In [33]:
y_names=le.classes_

In [91]:
y=pd.DataFrame(y,columns=['Fertilizer Name'])

In [92]:
y.to_csv('y.csv',index=False)

In [94]:
log_reg=LogisticRegression()
log_reg.fit(train,y)
log_reg.score(train,y)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.15952266666666667

In [103]:
xgbcl=xgb.XGBClassifier()
xgbcl.fit(train,y)
xgbcl.score(train,y)

0.24527466666666667

In [138]:
probs=xgbcl.predict_proba(test)

In [38]:
def get_sub_file(probs):
    output=np.argsort(probs, axis=1)[:, -3:][:, ::-1]
    output=y_names[output]
    output = [' '.join(map(str, row)) for row in output]
    submission = pd.DataFrame({'id': test_id, 'Fertilizer Name': output})
    submission.to_csv('submission.csv', index=False)

In [139]:
output=np.argsort(probs, axis=1)[:, -3:][:, ::-1]

In [140]:
output=xgbcl.classes_[output]

In [141]:
output=y_names[output]

In [142]:
output = [' '.join(map(str, row)) for row in output]

In [143]:
submission = pd.DataFrame({'id': test_id, 'Fertilizer Name': output})

In [144]:
submission.to_csv('submission.csv', index=False)

In [ ]:
xgb.XGBClassifier

In [4]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

# Parameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 6, 7, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [0.1, 0.5, 1.0, 1.5, 2.0],
    'min_child_weight': [1, 3, 5, 7]
}

# ✅ Instantiate the classifier
xgb_cl = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# ✅ Use the classifier instance, not the module
search = RandomizedSearchCV(
    estimator=xgb_cl,
    param_distributions=param_grid,
    n_iter=50,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Fit
search.fit(train, y)

# Output best parameters
print("Best Parameters:", search.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


KeyboardInterrupt: 

In [11]:
y.value_counts()

,count
Fertilizer Name,
1,114436
0,113887
2,112453
4,111158
3,110889
5,94860
6,92317


In [52]:
model=Sequential()
model.add(Dense(128,activation='relu',input_shape=(train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7,activation='softmax'))

early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train,y,epochs=100,batch_size=32,validation_split=0.2,callbacks=[early_stopping])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 63s 3ms/step - accuracy: 0.1516 - loss: 1.9433 - val_accuracy: 0.1530 - val_loss: 1.9412
Epoch 2/100
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 79s 3ms/step - accuracy: 0.1516 - loss: 1.9416 - val_accuracy: 0.1509 - val_loss: 1.9410
Epoch 3/100
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 58s 3ms/step - accuracy: 0.1513 - loss: 1.9412 - val_accuracy: 0.1531 - val_loss: 1.9406
Epoch 4/100
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 57s 3ms/step - accuracy: 0.1511 - loss: 1.9414 - val_accuracy: 0.1506 - val_loss: 1.9410
Epoch 5/100
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 58s 3ms/step - accuracy: 0.1518 - loss: 1.9409 - val_accuracy: 0.1507 - val_loss: 1.9411
Epoch 6/100
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.1524 - loss: 1.9413 - val_accuracy: 0.1526 - val_loss: 1.9406
Epoch 7/100
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.1521 - loss: 1.9410 - val_accuracy: 0.1526 - val_loss: 1.9407
Epoch 8/100
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 81s 3ms/step - ac

In [53]:
probs=model.predict(test)

7813/7813 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step


In [54]:
top3=np.argsort(probs, axis=1)[:, -3:][:, ::-1]
top3_probs=np.take_along_axis(probs, top3, axis=1)
output=y_names[top3]

In [55]:
get_sub_file(top3_probs)

In [45]:
output.shape

(187973, 3)

In [51]:
test.shape

(250000, 20)